# Tinder Data Extraction
## Author: Oliver Gladfelter
### 3/18/2020

In [3]:
import pandas as pd
import json
import csv 
import os
import datetime

def dayOfWeek(date):
    """
    Date is in YYYY-MM-DD format
    """
    dateParts = date.split("-")
    return datetime.datetime(int(dateParts[0]), int(dateParts[1]), int(dateParts[2])).strftime('%A')

def getTinderStats(path):
    # open and save json file
    with open(path, encoding="utf8") as json_file:
        data = json.load(json_file)

    # save user info
    gender = data["User"]["gender"]
    interested_in = data["User"]["interested_in"]
    age_filter_min = data["User"]["age_filter_min"]
    age_filter_max = data["User"]["age_filter_max"]

    # save swipe info
    swipes_likes = data["Usage"]["swipes_likes"]
    swipes_passes = data["Usage"]["swipes_passes"]
    matches = data["Usage"]["matches"]

    df = pd.DataFrame([swipes_likes,swipes_passes,matches], index = ["swipes_likes", "swipes_passes", "matches"]).transpose().reset_index()

    df['total_swipes'] = df['swipes_likes'] + df["swipes_passes"]

    # get summary totals
    summaryTotals = df.sum()
    swipes_likes = summaryTotals[1]  
    swipes_passes = summaryTotals[2] 
    matches = summaryTotals[3] 
    total_swipes = summaryTotals[4] 

    # access number of matches messaged with
    numMatchesMessagedWith = len(data["Messages"])

    # make df - only run if you want to OVERWRITE tinderStats.csv
    # tinderStats = pd.DataFrame({'name': name, 'gender':[gender], 'interested_in':[interested_in], 'age_filter_min':[age_filter_min], 'age_filter_max':[age_filter_max], 'total_swipes':[total_swipes], 'swipes_likes':[swipes_likes], 'swipes_passes':[swipes_passes], 'matches':[matches], 'matchesMessagedWith':[numMatchesMessagedWith]})
    # tinderStats.to_csv("tinderStats.csv", index=False) 
    
    # open tinderStats.csv and add row with tinder data
    with open(r'tinderStats.csv', 'a', newline='') as csvfile:
        fieldnames = ['gender', 'interested_in', 'age_filter_min', 'age_filter_max', 'total_swipes','swipes_likes', 'swipes_passes', 'matches', 'matchesMessagedWith']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writerow({'gender':str(gender), 'interested_in':str(interested_in), 'age_filter_min':str(age_filter_min), 'age_filter_max':str(age_filter_max), 'total_swipes':str(total_swipes), 'swipes_likes':str(swipes_likes), 'swipes_passes':str(swipes_passes), 'matches':str(matches), 'matchesMessagedWith':str(numMatchesMessagedWith)})

# Extract And Add Data To Existing CSV File

In [16]:
for file in os.listdir("Friends Tinder Data"):
    getTinderStats("Friends Tinder Data/" + file)

# App Opens

In [34]:
def app_opens(path):
    # open and save json file
    with open(path, encoding="utf8") as json_file:
        data = json.load(json_file)
        
    # convert app opens to df, convert dates to day of week,groupby day of week and sum
    app_opens = pd.DataFrame([data["Usage"]["app_opens"]], index=['app_opens']).transpose().reset_index()
    app_opens['dayOfWeek'] = app_opens['index'].apply(dayOfWeek)
    app_opens = app_opens.groupby('dayOfWeek').sum().sort_values('app_opens', ascending=False).reset_index()
    
    app_opens['name'] = data["User"]['full_name']
    app_opens["%"] = app_opens["app_opens"] / app_opens["app_opens"].sum()
    
    return app_opens

In [35]:
# create empty df to fill in
app_opens_df = pd.DataFrame({"dayOfWeek":[], "app_opens":[], "name":[]})

# access each friend's data, add to df
for file in os.listdir("Friends Tinder Data"):
    app_opens_df = pd.concat([app_opens_df,app_opens("Friends Tinder Data/" + file)])

In [43]:
# determine most active day
app_opens_df.groupby('dayOfWeek').sum().sort_values("%")

,%,app_opens
dayOfWeek,,
Friday,1.553945,4037.0
Thursday,1.586133,3808.0
Wednesday,1.654730,3797.0
Saturday,1.681083,3949.0
Monday,1.756707,4292.0
Tuesday,1.762040,4186.0
Sunday,2.005362,4892.0
